In [110]:
import requests,configparser
def get_cookie():
    config = configparser.ConfigParser()
    config.read('secrets.txt')
    cookie = config['session_info']['cookie']
    return cookie
def get_inputs(day):
    cookie, day = get_cookie(), int(day)
    headers = {'session': cookie}
    url = f'https://adventofcode.com/2021/day/{day}/input'
    session = requests.Session()
    resp = session.get(url,cookies=headers)
    return resp.text.split('\n')[:-1]

In [111]:
# DAY1
def count_increases(data,depth):
    increases = 0
    for i in range(depth,len(data)):
        increases += 1 if data[i]>data[i-depth] else 0
    return increases

data_day1 = [int(x) for x in get_inputs(1) if x!='']
print(f'Part 1: {count_increases(data_day1,1)}')
print(f'Part 2: {count_increases(data_day1,3)}')

Part 1: 1482
Part 2: 1518


In [112]:
# DAY2
class Submarine():
    def __init__(self,h_pos,depth,has_aim=False,aim=0):
        self.h_pos = h_pos
        self.depth = depth
        self.has_aim = has_aim
        self.aim = aim
    def forward(self,x):
        self.h_pos += int(x)
        self.depth += int(x)*self.aim
    def down(self,x):
        if self.has_aim:
            self.aim += int(x)
        else:
            self.depth += int(x)
    def up(self,x):
        if self.has_aim:
            self.aim -= int(x)
        else:
            self.depth -= int(x)
    def take_command(self,command):
        command_map = {'up': self.up, 'down':self.down, 'forward':self.forward}
        directions = command.split(' ')    # e.g. direction = ['forward','2']
        move_method = command_map.get(directions[0])
        move_method(directions[1])
    def get_position(self):
        return (self.h_pos,self.depth)
    def navigate(self,data):
        for command in data:
            self.take_command(command)
        return self
    def get_pos_multip(self):
        print( self.h_pos*self.depth )

data_day2 = get_inputs(2)
d2p1 = Submarine(0,0)
d2p1.navigate(data_day2).get_pos_multip()

d2p2 = Submarine(0,0,True,0)
d2p2.navigate(data_day2).get_pos_multip()

1938402
1947878632
